In [2]:
import os
os.getcwd()

'C:\\Users\\Precious Olalere\\Documents\\IS 567 Text Mining'

In [3]:
import pandas as pd
#Task 1: Importing the train and test da
#Listing the number of statements in the training set and the test set

drug_train = pd.read_csv("drugs_train.csv")
drug_test = pd.read_csv("drugs_test.csv")


print ("Train set: ",len(drug_train))
print ("Test set: ",len(drug_test))

drug_train.head(10)

Train set:  8066
Test set:  2690


,rating,Unnamed: 1,uniqueID,drugName,condition,review,rating.1,date,usefulCount
0,1,47336,232258,Belsomra,Insomnia,"""I started Belsomra 15mg about 3 wks ago. It h...",1,10-Jul-17,28
1,1,41429,31475,Miconazole,Vaginal Yeast Infection,"""Okay so I was having some symptoms of a yeast...",1,7-Jun-16,4
2,1,114929,15704,Ethinyl estradiol / norethindrone,Birth Control,"""I started taking the this pill and after a li...",1,20-Oct-16,11
3,1,109151,40272,Savella,ibromyalgia,"""Caused suicidal thoughts.""",1,31-Aug-13,47
4,1,130901,48056,Cariprazine,Bipolar Disorde,"""Started taking it today, 1.5mg, hour later he...",1,8-Jul-16,11
5,1,68436,151573,Chantix,Smoking Cessation,"""Ok- I have tried many things to quit smoking ...",1,10-Oct-15,9
6,1,117117,209214,Pentazocine,Pain,"""Scary medicine given for migraine, sent home....",1,10-Jan-13,4
7,1,90372,124443,Skyla,Birth Control,"""I had the Skyla inserted Friday and removed i...",1,5-Oct-15,1
8,1,36845,54585,Bupropion / naltrexone,Obesity,"""I took it for five months every day , I did n...",1,30-May-17,0
9,1,155572,174444,Tiotropium,"COPD, Maintenance","""Was prescribed Spiriva when ill with bronchit...",1,26-Jun-15,77


In [4]:
drug_test.head(10)

,rating,Unnamed: 1,uniqueID,drugName,condition,review,rating.1,date,usefulCount
0,1,49726,80865,Delsym,Cough,"""Took this medicine at 7:00pm, recommended by ...",1,1-Feb-17,1
1,1,42716,29970,Klonopin,Bipolar Disorde,"""Horrible side effects.""",1,19-Aug-16,11
2,1,45680,213832,Tioconazole,Vaginal Yeast Infection,"""They should really put a warning on the label...",1,21-Jan-17,15
3,1,50602,137036,Isotretinoin,Acne,"""I just started my Accutane today. My biggest ...",1,18-Jan-16,12
4,1,39804,97703,Simvastatin,High Cholesterol,"""Took simvastatin only once and had a bad case...",1,30-Dec-10,35
5,1,36577,107483,Implanon,Birth Control,"""I have Implanon since June 2010. You really d...",1,17-Mar-11,2
6,1,51437,42848,Xulane,Birth Control,"""This is horrible i never used to get cramps a...",1,13-Aug-16,5
7,1,52472,178140,Duloxetine,Depression,"""I was on Cymbalta for about 3 months. I can&...",1,15-Dec-10,13
8,1,3477,51231,Azithromycin,Chlamydia Infection,"""When I took this, I took it without food, and...",1,15-Nov-15,13
9,1,33953,32343,Lisinopril,Heart Attack,"""Having been taken to the hospital for an aryt...",1,15-Apr-16,13


In [5]:
#Convert the ratings to negative, neutral, and positive to make it a three-way classification for later tasks

#Group labels {1,2,3,4} -> negative, {5,6} -> neutral, {7,8,9,10} -> positive

group_label = {
    1: "negative",
    2: "negative",
    3: "negative",
    4: "negative",
    5: "neutral",
    6: "neutral",
    7: "positive",
    8: "positive",
    9: "positive",
    10: "positive"
}

# Apply the mapping to both the training and test datasets
drug_train["three_way_rating"] = drug_train["rating.1"].map(group_label)
drug_test["three_way_rating"] = drug_test["rating.1"].map(group_label)
print(drug_train[["rating.1", "three_way_rating"]].head())
print(drug_test[["rating.1", "three_way_rating"]].head())


   rating.1 three_way_rating
0         1         negative
1         1         negative
2         1         negative
3         1         negative
4         1         negative
   rating.1 three_way_rating
0         1         negative
1         1         negative
2         1         negative
3         1         negative
4         1         negative


In [7]:
# use original reviews for model building
y_train = drug_train["three_way_rating"]
y_test = drug_test["three_way_rating"]

train_text = drug_train["review"]
test_text = drug_test["review"]

In [8]:
#Extract unigrams from the review column in the training set. 
#Fit the unigrams to the review column in the training set and testing set. Note that we are using count-based features.

from sklearn.feature_extraction.text import CountVectorizer

# set the n-gram range
vectorizer = CountVectorizer(ngram_range = (1,1))

# create training data representation
train_data_cv = vectorizer.fit_transform(train_text.values.astype('U'))
print(train_data_cv.shape,"\n") 

# create test data representation
test_data_cv = vectorizer.transform(test_text.values.astype('U'))
print(test_data_cv.shape,"\n") 



(8066, 15185) 

(2690, 15185) 



In [9]:
# import relevant libraries
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, auc
from sklearn import preprocessing

In [11]:
#Train a classifier using the Naive Bayes algorithm (MultinomialNB). 
#Run the model on the test set and report precision, recall, and F1 score (micro, macro, and weighted average) of the classifier.

# build model on the training data
naive_bayes = MultinomialNB()
naive_bayes.fit(train_data_cv, y_train)
predictions = naive_bayes.predict(test_data_cv)

# average could be of 3 kinds = weighted, macro, micro
print("Accuracy score: ", accuracy_score(y_test, predictions))
print("Precision score: ", precision_score(y_test, predictions, average="weighted"))
print("Recall score: ", recall_score(y_test, predictions, average = "weighted"))
print("F1 score: ", f1_score(y_test, predictions, average = "weighted"))


Accuracy score:  0.7468401486988847
Precision score:  0.7087006024284891
Recall score:  0.7468401486988847
F1 score:  0.7067929598142093


In [12]:
# weighted average 
print("Accuracy score: ", accuracy_score(y_test, predictions))
print("Precision score: ", precision_score(y_test, predictions, average="weighted"))
print("Recall score: ", recall_score(y_test, predictions, average = "weighted"))
print("F1 score: ", f1_score(y_test, predictions, average = "weighted"))

Accuracy score:  0.7468401486988847
Precision score:  0.7087006024284891
Recall score:  0.7468401486988847
F1 score:  0.7067929598142093


In [13]:
# macro average 
print("Accuracy score: ", accuracy_score(y_test, predictions))
print("Precision score: ", precision_score(y_test, predictions, average="macro"))
print("Recall score: ", recall_score(y_test, predictions, average = "macro"))
print("F1 score: ", f1_score(y_test, predictions, average = "macro"))


Accuracy score:  0.7468401486988847
Precision score:  0.6054844787106601
Recall score:  0.49435712817038774
F1 score:  0.49094076196831615


In [15]:
# micro average 
print("Accuracy score: ", accuracy_score(y_test, predictions))
print("Precision score: ", precision_score(y_test, predictions, average="micro"))
print("Recall score: ", recall_score(y_test, predictions, average = "micro"))
print("F1 score: ", f1_score(y_test, predictions, average = "micro"))

Accuracy score:  0.7468401486988847
Precision score:  0.7468401486988847
Recall score:  0.7468401486988847
F1 score:  0.7468401486988847


In [16]:
#Task 2: Remove features that have variance values below the thresholds 0.001 and 0.005, respectively.
#low variance removal 
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold = 0.001)

X_train_features_filtered_var_thr = selector.fit(train_data_cv).transform(train_data_cv)
print ("Train feature space before filtering: ", train_data_cv.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_var_thr.shape)

X_test_features_filtered_var_thr = selector.transform(test_data_cv)
print ("Test feature space before filtering: ", test_data_cv.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_var_thr.shape)

Train feature space before filtering:  (8066, 15185)
Train feature space after filtering:  (8066, 3945)
Test feature space before filtering:  (2690, 15185)
Test feature space after filtering:  (2690, 3945)


In [17]:
#number 2 low variance removal for 0.005
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold = 0.005)

X_train_features_filtered_var_thr_two = selector.fit(train_data_cv).transform(train_data_cv)
print ("Train feature space before filtering: ", train_data_cv.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_var_thr_two.shape)

X_test_features_filtered_var_thr_two = selector.transform(test_data_cv)
print ("Test feature space before filtering: ", test_data_cv.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_var_thr_two.shape)

Train feature space before filtering:  (8066, 15185)
Train feature space after filtering:  (8066, 1546)
Test feature space before filtering:  (2690, 15185)
Test feature space after filtering:  (2690, 1546)


In [21]:
#repeat task 1 for task 2 (feature selection1)
#training Naive Bayes on low variance of 0.001
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_features_filtered_var_thr, y_train)
predictions = naive_bayes.predict(X_test_features_filtered_var_thr)

# weighted average 
print("Accuracy score: ", accuracy_score(y_test, predictions))
print("Precision score: ", precision_score(y_test, predictions, average="weighted"))
print("Recall score: ", recall_score(y_test, predictions, average = "weighted"))
print("F1 score: ", f1_score(y_test, predictions, average = "weighted"))

# macro average 
print("Accuracy score: ", accuracy_score(y_test, predictions))
print("Precision score: ", precision_score(y_test, predictions, average="macro"))
print("Recall score: ", recall_score(y_test, predictions, average = "macro"))
print("F1 score: ", f1_score(y_test, predictions, average = "macro"))


#micro average 
print("Accuracy score: ", accuracy_score(y_test, predictions))
print("Precision score: ", precision_score(y_test, predictions, average="micro"))
print("Recall score: ", recall_score(y_test, predictions, average = "micro"))
print("F1 score: ", f1_score(y_test, predictions, average = "micro"))

Accuracy score:  0.7211895910780669
Precision score:  0.7027812906429134
Recall score:  0.7211895910780669
F1 score:  0.7110796473919929
Accuracy score:  0.7211895910780669
Precision score:  0.5352856004116763
Recall score:  0.5282459212439216
F1 score:  0.529086309432013
Accuracy score:  0.7211895910780669
Precision score:  0.7211895910780669
Recall score:  0.7211895910780669
F1 score:  0.7211895910780669


In [22]:
#repeat task 1 for task 2 
#training Naive Bayes on low variance of 0.005
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_features_filtered_var_thr_two, y_train)
predictions = naive_bayes.predict(X_test_features_filtered_var_thr_two)

# weighted average
print("Accuracy score(weighted): ", accuracy_score(y_test, predictions))
print("Precision score(weighted): ", precision_score(y_test, predictions, average="weighted"))
print("Recall score(weighted): ", recall_score(y_test, predictions, average = "weighted"))
print("F1 score(weighted): ", f1_score(y_test, predictions, average = "weighted"))

# macro average
print("Accuracy score(macro): ", accuracy_score(y_test, predictions))
print("Precision score(macro): ", precision_score(y_test, predictions, average="macro"))
print("Recall score(macro): ", recall_score(y_test, predictions, average = "macro"))
print("F1 score(macro): ", f1_score(y_test, predictions, average = "macro"))

# micro average
print("Accuracy score(micro): ", accuracy_score(y_test, predictions))
print("Precision score(micro): ", precision_score(y_test, predictions, average="micro"))
print("Recall score(micro): ", recall_score(y_test, predictions, average = "micro"))
print("F1 score(micro): ", f1_score(y_test, predictions, average = "micro"))



Accuracy score(weighted):  0.7011152416356877
Precision score(weighted):  0.6963765085265881
Recall score(weighted):  0.7011152416356877
F1 score(weighted):  0.6986782488657574
Accuracy score(macro):  0.7011152416356877
Precision score(macro):  0.5269476278257468
Recall score(macro):  0.5235315468171952
F1 score(macro):  0.5251348961296606
Accuracy score(micro):  0.7011152416356877
Precision score(micro):  0.7011152416356877
Recall score(micro):  0.7011152416356877
F1 score(micro):  0.7011152416356877


#Report how thresholds affect performance. 

The threshold performance for 0.001 reduced the features from 15185 to 3945 and the threshold of 0.005 even reduced the feature even more at 1546 features. 

#Discuss the relationship between the number of features after feature selection and the performance of the resulting model.

The performance of the resulting model after the low variance removal definitely improved the model compared to when the model had a lot of features. 


In [23]:
#TASK 3
#Calculate the “informativeness” of the original features using mutual information. 
#Select the top k features based on their informativeness values with k = 1000 and 2000. 
#feature selection with chi-squared statistics and k-best features 

from sklearn.feature_selection import SelectKBest, chi2

selector = SelectKBest(chi2, k=1000)
X_train_features_filtered_kbest = selector.fit_transform(train_data_cv, y_train)
print ("Train feature space before filtering: ", train_data_cv.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_kbest.shape)

X_test_features_filtered_kbest = selector.transform(test_data_cv)
print ("Test feature space before filtering: ", test_data_cv.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_kbest.shape)

Train feature space before filtering:  (8066, 15185)
Train feature space after filtering:  (8066, 1000)
Test feature space before filtering:  (2690, 15185)
Test feature space after filtering:  (2690, 1000)


In [24]:
# number 2:K features of 2000
from sklearn.feature_selection import SelectKBest, chi2

selector = SelectKBest(chi2, k=2000)
X_train_features_filtered_kbest_two = selector.fit_transform(train_data_cv, y_train)
print ("Train feature space before filtering: ", train_data_cv.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_kbest_two.shape)

X_test_features_filtered_kbest_two = selector.transform(test_data_cv)
print ("Test feature space before filtering: ", test_data_cv.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_kbest_two.shape)

Train feature space before filtering:  (8066, 15185)
Train feature space after filtering:  (8066, 2000)
Test feature space before filtering:  (2690, 15185)
Test feature space after filtering:  (2690, 2000)


In [25]:
#repeating model building for task 
#repeat task 1 for task 3 (k-best feature)
#training Naive Bayes on k-best feature 0f 1000
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_features_filtered_kbest, y_train)
predictions = naive_bayes.predict(X_test_features_filtered_kbest)

# weighted average
print("Accuracy score(weighted): ", accuracy_score(y_test, predictions))
print("Precision score(weighted): ", precision_score(y_test, predictions, average="weighted"))
print("Recall score(weighted): ", recall_score(y_test, predictions, average = "weighted"))
print("F1 score(weighted): ", f1_score(y_test, predictions, average = "weighted"))

# macro average
print("Accuracy score(macro): ", accuracy_score(y_test, predictions))
print("Precision score(macro): ", precision_score(y_test, predictions, average="macro"))
print("Recall score(macro): ", recall_score(y_test, predictions, average = "macro"))
print("F1 score(macro): ", f1_score(y_test, predictions, average = "macro"))

# micro average
print("Accuracy score(micro): ", accuracy_score(y_test, predictions))
print("Precision score(micro): ", precision_score(y_test, predictions, average="micro"))
print("Recall score(micro): ", recall_score(y_test, predictions, average = "micro"))
print("F1 score(micro): ", f1_score(y_test, predictions, average = "micro"))



Accuracy score(weighted):  0.7271375464684015
Precision score(weighted):  0.6872056125483135
Recall score(weighted):  0.7271375464684015
F1 score(weighted):  0.703364825303648
Accuracy score(macro):  0.7271375464684015
Precision score(macro):  0.5175340061617322
Recall score(macro):  0.5043399176609452
F1 score(macro):  0.5008649731210002
Accuracy score(micro):  0.7271375464684015
Precision score(micro):  0.7271375464684015
Recall score(micro):  0.7271375464684015
F1 score(micro):  0.7271375464684015


In [26]:
#repeating model building for task 
#repeat task 1 for task 3 (k-best feature)
#training Naive Bayes on k-best feature 0f 2000
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_features_filtered_kbest_two, y_train)
predictions = naive_bayes.predict(X_test_features_filtered_kbest_two)

# weighted average
print("Accuracy score(weighted): ", accuracy_score(y_test, predictions))
print("Precision score(weighted): ", precision_score(y_test, predictions, average="weighted"))
print("Recall score(weighted): ", recall_score(y_test, predictions, average = "weighted"))
print("F1 score(weighted): ", f1_score(y_test, predictions, average = "weighted"))

# macro average
print("Accuracy score(macro): ", accuracy_score(y_test, predictions))
print("Precision score(macro): ", precision_score(y_test, predictions, average="macro"))
print("Recall score(macro): ", recall_score(y_test, predictions, average = "macro"))
print("F1 score(macro): ", f1_score(y_test, predictions, average = "macro"))

# micro average
print("Accuracy score(micro): ", accuracy_score(y_test, predictions))
print("Precision score(micro): ", precision_score(y_test, predictions, average="micro"))
print("Recall score(micro): ", recall_score(y_test, predictions, average = "micro"))
print("F1 score(micro): ", f1_score(y_test, predictions, average = "micro"))




Accuracy score(weighted):  0.7271375464684015
Precision score(weighted):  0.6902360035756264
Recall score(weighted):  0.7271375464684015
F1 score(weighted):  0.7044584818449964
Accuracy score(macro):  0.7271375464684015
Precision score(macro):  0.5306166558598101
Recall score(macro):  0.5078742508922481
F1 score(macro):  0.507901482510225
Accuracy score(micro):  0.7271375464684015
Precision score(micro):  0.7271375464684015
Recall score(micro):  0.7271375464684015
F1 score(micro):  0.7271375464684015


#Report how the k value affects performance. 
#Discuss the relationship between the number of features after feature selection and the performance of the resulting model.

The features reduced based on the value of the k-value that was assigned to it, the informativeness value of 1000, reduced the feature to 1000 and the informativeness value of 2000 reduced the features to 2000. 

The k-value affected the performance of the model by reducing the accuracy, precision, recall and F1 score compared to the other models 

In [27]:
##TASK 4(feature selection 3: Lexicon-based feature selection)
# loading in the postive word list
with open("positive_words_list.txt", 'r') as f:
    positive_words = [line.strip() for line in f]
print("words list:", positive_words[:10])


words list: ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']


In [28]:
# loading in the negative word list
with open("negative_words_list.txt", 'r') as f:
    negative_words = [line.strip() for line in f]
print("words list:", negative_words[:10])


words list: ['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']


In [29]:
#TASK 4(feature selection 3: Lexicon-based feature selection)
# Joining both postive and negative word list into one single list

lexicon = set (positive_words + negative_words)
print("lexicon list:", lexicon)



lexicon list: {'flaky', 'friggin', 'cooperative', 'unwilling', 'spite', 'worried', 'self-respect', 'prohibitively', 'peaceful', 'frustrate', 'ignore', 'lapse', 'remorsefully', 'concerned', 'reverently', 'noises', 'fat', 'satisfactory', 'unfortunate', 'abyss', 'disintegrates', 'officious', 'disputable', 'fretful', 'masters', 'enlightenment', 'stupor', 'uproarious', 'phobic', 'kaput', 'diversified', 'tense', 'swift', 'exploitation', 'belie', 'protect', 'sprightly', 'propagandize', 'meager', 'degradation', 'prevaricate', 'astonishingly', 'unknown', 'nightmarishly', 'biases', 'cancerous', 'reform', 'slanderously', 'harshly', 'admiring', 'disadvantaged', 'regretful', 'luxuriously', 'blurred', 'inept', 'blistering', 'passionate', 'detraction', 'regal', 'asininely', 'criticizing', 'primitive', 'blunder', 'combative', 'wondrous', 'abnormal', 'jolly', 'temptingly', 'bruises', 'efficient', 'raptureously', 'scary', 'sagging', 'reckless', 'winning', 'scream', 'virulent', 'prosperous', 'superficial

In [30]:
#Use the words in positive and negative word lists to extract unigram features from the dataset

from sklearn.feature_extraction.text import CountVectorizer
# set the n-gram range
vectorizer = CountVectorizer(ngram_range = (1,1), vocabulary=lexicon)

# create training data representation
train_data_cv = vectorizer.fit_transform(train_text.values.astype('U'))
print(train_data_cv.shape,"\n") 

# create test data representation
test_data_cv = vectorizer.transform(test_text.values.astype('U'))
print(test_data_cv.shape,"\n") 


(8066, 6786) 

(2690, 6786) 



In [31]:
#Train a model that uses these lexicon-based features.  
#Report the number of features in the training set and the test set.
# build model on the training data
#training Naive Bayes on lexicon-based features
naive_bayes = MultinomialNB()
naive_bayes.fit(train_data_cv, y_train)
predictions = naive_bayes.predict(test_data_cv)

# weighted average
print("Accuracy score(weighted): ", accuracy_score(y_test, predictions))
print("Precision score(weighted): ", precision_score(y_test, predictions, average="weighted", zero_division=0))
print("Recall score(weighted): ", recall_score(y_test, predictions, average = "weighted", zero_division=0))
print("F1 score(weighted): ", f1_score(y_test, predictions, average = "weighted", zero_division=0))

# macro average
print("Accuracy score(macro): ", accuracy_score(y_test, predictions))
print("Precision score(macro): ", precision_score(y_test, predictions, average="macro", zero_division=0))
print("Recall score(macro): ", recall_score(y_test, predictions, average = "macro", zero_division=0))
print("F1 score(macro): ", f1_score(y_test, predictions, average = "macro", zero_division=0))

# micro average
print("Accuracy score(micro): ", accuracy_score(y_test, predictions))
print("Precision score(micro): ", precision_score(y_test, predictions, average="micro", zero_division=0))
print("Recall score(micro): ", recall_score(y_test, predictions, average = "micro", zero_division=0))
print("F1 score(micro): ", f1_score(y_test, predictions, average = "micro", zero_division=0))



Accuracy score(weighted):  0.7029739776951673
Precision score(weighted):  0.6717887232406905
Recall score(weighted):  0.7029739776951673
F1 score(weighted):  0.6374023444581838
Accuracy score(macro):  0.7029739776951673
Precision score(macro):  0.612597421592595
Recall score(macro):  0.4153912814241994
F1 score(macro):  0.4065626274742628
Accuracy score(micro):  0.7029739776951673
Precision score(micro):  0.7029739776951673
Recall score(micro):  0.7029739776951673
F1 score(micro):  0.7029739776951673


In [80]:
#Task 6:Use mutual information to select the best 2000 unigram features
# use original reviews for model building
y_train = drug_train["three_way_rating"]
y_test = drug_test["three_way_rating"]

train_text = drug_train["review"]
test_text = drug_test["review"]

In [81]:
cv= CountVectorizer(ngram_range = (1,1))

X_train_cv=cv.fit_transform(train_text)
X_test_cv=cv.transform(test_text)
selector = SelectKBest(chi2, k=2000)
X_train_features_filtered_kbest_k = selector.fit_transform(X_train_cv, y_train)
print ("Train feature space before filtering: ", train_text.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_kbest_k.shape)

X_test_features_filtered_kbest_k = selector.transform(X_test_cv)
print ("Test feature space before filtering: ", test_text.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_kbest_k.shape)

Train feature space before filtering:  (8066,)
Train feature space after filtering:  (8066, 2000)
Test feature space before filtering:  (2690,)
Test feature space after filtering:  (2690, 2000)


In [82]:
#Task 7,8 and 9: Train and evaluate a Naive Bayes classifier using cross-validation
#Train a linear SVM classifier 
#Train a logistic regression classifier 
#Using only the training set, train a Naive Bayes classifier (MultinomialNB) using 5-fold cross-validation.
#Use the unigram features obtained in Task 6.
#Model comparison across K-fold cross validation
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score
#from sklearn.model_selection import cross_validate
import math
from scipy import stats

from sklearn.model_selection import StratifiedKFold, cross_val_score
import numpy as np

skf = StratifiedKFold(n_splits=5)

#print(skf)
metrics_svm=[]
metrics_lr=[]
metrics_nb=[]
report_nb = []
report_svm=[]
report_lr = []

for train_index, test_index in skf.split(X_train_features_filtered_kbest_k, y_train):
    print("Training instance indexes:", train_index)
    print("Test instance indexes:", test_index)
    X_train = X_train_features_filtered_kbest_k[train_index]
    X_test = X_train_features_filtered_kbest_k[test_index]
    Y_train = y_train[train_index]
    Y_test = y_train[test_index]

    # for naivesbayes model
    naive_bayes = MultinomialNB()
    naive_bayes.fit(X_train, Y_train)
    predictions_nb = naive_bayes.predict(X_test)
    
    accuracy_nb = accuracy_score(Y_test, predictions_nb)
    precision_nb = precision_score(Y_test, predictions_nb, average='weighted')
    recall_nb = recall_score(Y_test, predictions_nb, average='weighted')
    f1_nb = f1_score(Y_test, predictions_nb, average = "weighted")
    report_nb.append((accuracy_nb, precision_nb, recall_nb, f1_nb))

    # for linear SVM
    svm_classifier = SVC(kernel='linear')
    svm_classifier.fit(X_train, Y_train)
    predictions_svm = svm_classifier.predict(X_test)
    
    accuracy_svm = accuracy_score(Y_test, predictions_svm)
    precision_svm = precision_score(Y_test, predictions_svm, average='weighted')
    recall_svm = recall_score(Y_test, predictions_svm, average='weighted')
    f1_svm = f1_score(Y_test, predictions_svm, average='weighted')
    metrics_svm.append((accuracy_svm, precision_svm, recall_svm, f1_svm))
    
    #for logistic regression
    LR = LogisticRegression(solver="liblinear", penalty="l1")
    LR.fit(X_train, Y_train)
    predictions_lr = LR.predict(X_test)

    accuracy_lr = accuracy_score(Y_test, predictions_lr)
    precision_lr = precision_score(Y_test, predictions_lr, average='weighted')
    recall_lr = recall_score(Y_test, predictions_lr, average='weighted')
    f1_lr = f1_score(Y_test, predictions_lr, average='weighted')
    metrics_lr.append((accuracy_lr, precision_lr, recall_lr, f1_lr))
    
    print(f"Report: \n"
          f"Naive Bayes -> Accuracy: {accuracy_nb}, Precision: {precision_nb}, Recall: {recall_nb}, F1: {f1_nb}\n"
          f"SVM -> Accuracy: {accuracy_svm}, Precision: {precision_svm}, Recall: {recall_svm}, F1: {f1_svm}\n"
          f"Logistic Regression -> Accuracy: {accuracy_lr}, Precision: {precision_lr}, Recall: {recall_lr}, F1: {f1_lr}\n")

Training instance indexes: [ 401  402  403 ... 8063 8064 8065]
Test instance indexes: [   0    1    2 ... 3789 3790 3791]
Report: 
Naive Bayes -> Accuracy: 0.7521685254027262, Precision: 0.7504113554824967, Recall: 0.7521685254027262, F1: 0.7418881657358843
SVM -> Accuracy: 0.7149938042131351, Precision: 0.718560616420412, Recall: 0.7149938042131351, F1: 0.7124716544232282
Logistic Regression -> Accuracy: 0.7323420074349443, Precision: 0.7190939197948962, Recall: 0.7323420074349443, F1: 0.7213322729433755

Training instance indexes: [   0    1    2 ... 8063 8064 8065]
Test instance indexes: [ 401  402  403 ... 4857 4858 4859]
Report: 
Naive Bayes -> Accuracy: 0.7861128332300062, Precision: 0.7785446588561016, Recall: 0.7861128332300062, F1: 0.7729673562337255
SVM -> Accuracy: 0.7482951022938623, Precision: 0.7280344618765338, Recall: 0.7482951022938623, F1: 0.7360783569933526
Logistic Regression -> Accuracy: 0.7699938003719777, Precision: 0.7418247414082584, Recall: 0.7699938003719777,

In [83]:
#Calculating the 95% confidence interval of the accuracy score 
#from the cross-validation results of naives bayes, SVM and Logisitic regression models

# define function to calculate the confidence interval
def calculate_confidence_interval(accuracy_scores):
    mean_accuracy = np.mean(accuracy_scores)
    std_accuracy = np.std(accuracy_scores, ddof=1)  
    n = len(accuracy_scores)  
    z = 1.959 
    margin_of_error = z * (std_accuracy / np.sqrt(n))
    confidence_interval = (mean_accuracy - margin_of_error, mean_accuracy + margin_of_error)
    return confidence_interval

# Extract accuracy scores for Naive Bayes, SVM, and Logistic Regression
accuracy_scores_nb = [score[0] for score in report_nb]  
accuracy_scores_svm = [score[0] for score in metrics_svm]  
accuracy_scores_lr = [score[0] for score in metrics_lr]  

# Calculate confidence intervals for each model
ci_nb = calculate_confidence_interval(accuracy_scores_nb)
ci_svm = calculate_confidence_interval(accuracy_scores_svm)
ci_lr = calculate_confidence_interval(accuracy_scores_lr)

print(f"95% Confidence Interval for Naive Bayes Accuracy: {ci_nb[0]:.4f} to {ci_nb[1]:.4f}")
print(f"95% Confidence Interval for SVM Accuracy: {ci_svm[0]:.4f} to {ci_svm[1]:.4f}")
print(f"95% Confidence Interval for Logistic Regression Accuracy: {ci_lr[0]:.4f} to {ci_lr[1]:.4f}")


95% Confidence Interval for Naive Bayes Accuracy: 0.7628 to 0.8053
95% Confidence Interval for SVM Accuracy: 0.7321 to 0.7705
95% Confidence Interval for Logistic Regression Accuracy: 0.7431 to 0.7779


In [87]:
#Task 10: Model comparison using paired t-test
#Use the ttest_ind function from scipy stats to compare the performance
#between classifier pairs (based on overall weighted F1 from cross-validation folds) with significance level alpha = 0.05
# Extract F1 scores from cross-validation folds for each model
f1_scores_nb = [score[3] for score in report_nb] 
f1_scores_svm = [score[3] for score in metrics_svm]  
f1_scores_lr = [score[3] for score in metrics_lr] 


# Significance level
alpha = 0.05

#Performing t-tests for all the models 
# LinearSVC vs. Naive Bayes
ttest_svm_nb = stats.ttest_rel(f1_scores_svm, f1_scores_nb)
# LinearSVC vs. Logistic Regression
ttest_svm_lr = stats.ttest_rel(f1_scores_svm, f1_scores_lr)
# Naive Bayes vs. Logistic Regression
ttest_nb_lr = stats.ttest_rel(f1_scores_nb, f1_scores_lr)

#printing the results
print("t-test result: ", ttest_svm_nb) 
print("t-test result: ", ttest_svm_lr)
print("t-test result: ", ttest_nb_lr)


t-test result:  TtestResult(statistic=-6.17576558304914, pvalue=0.003491787507960148, df=4)
t-test result:  TtestResult(statistic=-0.006472836077527954, pvalue=0.9951454153158164, df=4)
t-test result:  TtestResult(statistic=5.972383379612333, pvalue=0.003948719486132399, df=4)


In [88]:
#Results interpretation for Task 10
def interpret_ttest_result(ttest_result, model1, model2):
    t_stat, p_value = ttest_result
    if p_value < alpha:
        return f"Significant difference between {model1} and {model2} (p-value: {p_value:.4f})"
    else:
        return f"No significant difference between {model1} and {model2} (p-value: {p_value:.4f})"

# Printing results
print(interpret_ttest_result(ttest_svm_nb, "LinearSVC", "Naive Bayes"))
print(interpret_ttest_result(ttest_svm_lr, "LinearSVC", "Logistic Regression"))
print(interpret_ttest_result(ttest_nb_lr, "Naive Bayes", "Logistic Regression"))


Significant difference between LinearSVC and Naive Bayes (p-value: 0.0035)
No significant difference between LinearSVC and Logistic Regression (p-value: 0.9951)
Significant difference between Naive Bayes and Logistic Regression (p-value: 0.0039)


In [89]:
#Task 11: Hyperparameter tuning

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score
from sklearn.svm import LinearSVC

# Defining the parameter grid for C values
param_grid = {'C': [0.01, 0.1, 1, 10, 100, 1000]}

# Initializing the LinearSVC model
svm_classifier = LinearSVC(dual=False, max_iter=10000)
grid_search = GridSearchCV(svm_classifier, param_grid, cv=5, scoring='f1_weighted')  
grid_search.fit(X_train_features_filtered_kbest_k, y_train)

# Best regularization (C) value
best_C = grid_search.best_params_['C']
print(f"Best regularization parameter C: {best_C}")

# Using the best model to make predictions on the test set
best_model = grid_search.best_estimator_
y_pred_test = best_model.predict(X_test_features_filtered_kbest_k)  

# Calculating the weighted precision, recall, and F1 score for the best model
precision_test = precision_score(y_test, y_pred_test, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')

# Printing the results
print(f"Weighted Precision of the best model on test set: {precision_test:.4f}")
print(f"Weighted Recall of the best model on test set: {recall_test:.4f}")
print(f"Weighted F1 Score of the best model on test set: {f1_test:.4f}")


C:\Users\Precious Olalere\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Precious Olalere\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Precious Olalere\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Precious Olalere\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Precious Olalere\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Best regularization parameter C: 0.1
Weighted Precision of the best model on test set: 0.7112
Weighted Recall of the best model on test set: 0.7465
Weighted F1 Score of the best model on test set: 0.7179
